In [ ]:
from src.data_loader import load_data
from src.cleaning import handle_missing, remove_outliers
from src.eda import plot_time_series, plot_correlation


In [ ]:

df = load_data('data/benin.csv')


In [ ]:
numeric_cols = ['GHI','DNI','DHI','ModA','ModB','WS','WSgust']

# Fill missing values and remove outliers
df = handle_missing(df, numeric_cols)
df = remove_outliers(df, numeric_cols)


In [ ]:
df.to_csv('data/benin_clean.csv', index=False)


In [ ]:
# Time series of GHI
plot_time_series(df, 'Timestamp', 'GHI', 'GHI Over Time - Benin')

# Correlation heatmap
plot_correlation(df, numeric_cols)
